In [22]:
# --- SETUP & LOAD DATA ---

import pandas as pd
import sqlite3

file_name = 'Analisis Keterlambatan & Akurasi Waktu (Final).xlsx'
try:
    df = pd.read_excel(file_name)
    # Clean column names
    df.columns = [c.strip().replace(' ', '_').replace('.', '').replace('(', '').replace(')', '').replace('/', '_per_').lower() for c in df.columns]
    
    # Setup SQL Memory
    conn = sqlite3.connect(':memory:')
    df.to_sql('pizza_sales', conn, index=False, if_exists='replace')
    print("✅ Database Loaded Successfully\n")
except Exception as e:
    print(f"Error: {e}")

✅ Database Loaded Successfully



In [23]:
# --- EFFICIENCY ANALYSIS ---

query_efficiency = """
SELECT 
    restaurant_name,
    AVG(dist_km) as avg_distance_km,
    AVG(delivery_duration_min) as avg_duration,
    ROUND(SUM(delivery_duration_min) / SUM(dist_km), 2) as avg_efficiency_min_per_km,
    SUM(CASE WHEN is_delayed = 'Yes' THEN 1 ELSE 0 END) as total_late_orders
FROM 
    pizza_sales
GROUP BY 
    restaurant_name
HAVING 
    COUNT(order_id) > 5
ORDER BY 
    avg_efficiency_min_per_km ASC;
"""

print("--- EFISIENSI RESTORAN ---")
display(pd.read_sql_query(query_efficiency, conn))

--- EFISIENSI RESTORAN ---


,restaurant_name,avg_distance_km,avg_duration,avg_efficiency_min_per_km,total_late_orders
0,Domino's,5.750000,30.259434,5.0,0
1,Little Caesars,5.271357,28.844221,5.0,0
2,Marco's Pizza,5.189744,30.230769,5.0,0
3,Pizza Hut,5.000000,29.948454,5.0,0
4,Papa John's,4.387255,28.186275,6.0,0


In [19]:
# --- PRODUCT COMPLEXITY ANALYSIS ---

query_complexity = """
SELECT 
    pizza_complexity,
    pizza_size,
    COUNT(order_id) as total_orders,
    ROUND(AVG(toppings_count), 1) as avg_toppings,
    ROUND(AVG(delivery_duration_min), 1) as avg_delivery_time,
    ROUND(
        (SUM(CASE WHEN is_delayed = 'Yes' THEN 1 ELSE 0 END) * 100.0) / COUNT(order_id), 
        1
    ) as late_percentage
FROM 
    pizza_sales
GROUP BY 
    pizza_complexity, pizza_size
ORDER BY 
    late_percentage DESC;
"""

print("\n--- DAMPAK KOMPLEKSITAS PIZZA ---")
display(pd.read_sql_query(query_complexity, conn))


--- DAMPAK KOMPLEKSITAS PIZZA ---


,pizza_complexity,pizza_size,total_orders,avg_toppings,avg_delivery_time,late_percentage
0,1,Small,43,1.0,20.2,0.0
1,2,Small,87,2.0,22.5,0.0
2,3,Small,2,3.0,27.5,0.0
3,4,Medium,110,2.0,26.3,0.0
4,6,Medium,308,3.0,28.0,0.0
5,8,Medium,11,4.0,27.7,0.0
6,8,XL,1,2.0,45.0,0.0
7,9,Large,9,3.0,28.3,0.0
8,12,Large,229,4.0,34.3,0.0
9,15,Large,2,5.0,40.0,0.0


In [18]:
# --- OPERATIONAL BOTTLENECKS ---
query_window = """
SELECT 
    restaurant_name,
    order_id,
    delivery_duration_min,
    traffic_level,
    pizza_complexity,
    (delivery_duration_min - 30) as excess_time_minutes
FROM (
    SELECT 
        *,
        RANK() OVER (PARTITION BY restaurant_name ORDER BY delivery_duration_min DESC) as rank_lambat
    FROM 
        pizza_sales
) ranked_table 
WHERE 
    rank_lambat = 1
ORDER BY 
    delivery_duration_min DESC;
"""
print("\n--- OPERATIONAL BOTTLENECKS ---")
display(pd.read_sql_query(query_window, conn))


--- OPERATIONAL BOTTLENECKS ---


,restaurant_name,order_id,delivery_duration_min,traffic_level,pizza_complexity,excess_time_minutes
0,Domino's,ORD465,50,High,12,20
1,Domino's,ORD475,50,High,12,20
2,Domino's,ORD485,50,High,12,20
3,Domino's,ORD493,50,High,12,20
4,Domino's,ORD503,50,High,12,20
5,Domino's,ORD513,50,High,12,20
6,Domino's,ORD521,50,High,12,20
7,Domino's,ORD531,50,High,12,20
8,Domino's,ORD541,50,High,12,20
9,Domino's,ORD554,50,High,12,20
